# Generalized algebraic datatypes

#### references

- https://v2.ocaml.org/manual/gadts-tutorial.html#sec63
- https://sites.google.com/site/ocamlgadt/
- https://dev.realworldocaml.org/gadts.html
- https://www.cl.cam.ac.uk/teaching/1415/L28/gadts.pdf
- https://blog.mads-hartmann.com/ocaml/2015/01/05/gadt-ocaml.html
- https://blog.janestreet.com/why-gadts-matter-for-performance/
- https://stackoverflow.com/questions/27864200/an-concrete-simple-example-to-demonstrate-gadt-in-ocaml
- http://homepage.cs.uiowa.edu/~astump/papers/icfp09.pdf
- https://ocamllabs.io/iocamljs/type_inference.html

In [16]:
type 'a succ = unit -> 'a 
and _ peano =
    | Zero : unit peano
    | Succ : 'a peano -> ('a succ) peano ;;
    
let zero = Zero ;;
let one = Succ zero ;;
let two = Succ one ;;
let three = Succ two ;;

let count p =
    let rec loop : type a. a peano -> int -> int = fun p c ->
        match p with
        | Zero -> c
        | Succ n -> loop n (c + 1)
    in
    loop p 0 ;;

count two ;;
count three ;;

type 'a succ = unit -> 'a
and _ peano = Zero : unit peano | Succ : 'a peano -> 'a succ peano


val zero : unit peano = Zero


val one : unit succ peano = Succ Zero


val two : unit succ succ peano = Succ (Succ Zero)


val three : unit succ succ succ peano = Succ (Succ (Succ Zero))


val count : 'a peano -> int = <fun>


- : int = 2


- : int = 3


In [81]:
module GList = struct 
    type zero
    and _ t =
        | [] :  zero t
        | ( :: ) : 'a * 'b t -> ('a * 'b) t
    
    let hd : type a. (a * _) t -> a = function (h :: _) -> h ;;  
    let tl : type a. (_ * a) t -> a t = function (_ :: tl ) -> tl ;;
    let rec length : type a. a t -> int = function
        | [] -> 0
        | _ :: t -> 1 + (length t) 
end ;;

let lst : (_*_) GList.t = ["hello"; 1.1; 1; 'a'] ;;
GList.hd lst ;;
GList.tl lst ;;
GList.length lst ;;
GList.hd ([] : (_ * _) GList.t) ;;

module GList :
  sig
    type zero
    and _ t = [] : zero t | (::) : 'a * 'b t -> ('a * 'b) t
    val hd : ('a * 'b) t -> 'a
    val tl : ('b * 'a) t -> 'a t
    val length : 'a t -> int
  end


val lst : (string * (float * (int * (char * GList.zero)))) GList.t =
  GList.(::) ("hello",
   GList.(::) (1.1, GList.(::) (1, GList.(::) ('a', GList.[]))))


- : string = "hello"


- : (float * (int * (char * GList.zero))) GList.t =
GList.(::) (1.1, GList.(::) (1, GList.(::) ('a', GList.[])))


- : int = 4


error: compile_error

In [223]:
module GList = struct 
    type zero
    type (_,_) t =
        | Nil :  ('a, zero) t
        | Cons : 'a * ('a, 'b) t -> ('a, unit) t
    
    let hd : type a. (a, unit) t -> a = function (Cons (h,_)) -> h ;;  
    
    (* todo: implement tl and length function *)
end ;;

let lst =  GList.Cons (1, Cons(2, Nil)) ;;
GList.hd lst ;;

(* GList.tl lst ;;
GList.length lst ;; *)
GList.hd ([] : (_,_) GList.t) ;;

module GList :
  sig
    type zero
    type (_, _) t =
        Nil : ('a, zero) t
      | Cons : 'a * ('a, 'b) t -> ('a, unit) t
    val hd : ('a, unit) t -> 'a
  end


val lst : (int, unit) GList.t = GList.Cons (1, GList.Cons (2, GList.Nil))


- : int = 1


error: compile_error

In [29]:
type _ value =
    | Int : int -> int value
    | Float : float -> float value
    | Bool : bool -> bool value ;;
    
type _ expr =
    | Value : 'a value -> 'a expr
    | Eq : int expr * int expr -> bool expr
    | Plus : int expr * int expr -> int expr
    | FPlus : float expr * float expr -> float expr
    | If :  bool expr * 'a expr * 'a expr -> 'a expr ;;
    
let eval_value : type a. a value -> a = 
    function Int x -> x | Float x -> x | Bool x -> x ;;
    
let rec eval : type a. a expr -> a = 
 fun e : a ->    
    match e with
    | Value v -> eval_value v 
    | Eq (x, y) -> (eval x) = (eval y)
    | Plus (x, y) -> (eval x) + (eval y)
    | FPlus (x, y) -> (eval x) +. (eval y)
    | If (c, x, y) -> if eval c then eval x else eval y ;;
    
eval (FPlus (Value (Float 10.), Value (Float 10.))) ;;

type _ value =
    Int : int -> int value
  | Float : float -> float value
  | Bool : bool -> bool value


type _ expr =
    Value : 'a value -> 'a expr
  | Eq : int expr * int expr -> bool expr
  | Plus : int expr * int expr -> int expr
  | FPlus : float expr * float expr -> float expr
  | If : bool expr * 'a expr * 'a expr -> 'a expr


val eval_value : 'a value -> 'a = <fun>


val eval : 'a expr -> 'a = <fun>


- : float = 20.
